In [1]:
def GUI_vrep(x,y):
    x = x - 555
    y = y - 505
    
    x = x/100
    y = y/100
    
    return(x,y)
    

In [2]:
def vrep_GUI(x,y):
    x = x*100
    y = y*100
    
    if x < 0:
        x = 1110 - int(round(abs(x) + 555))
    else:
        x = int(round(abs(x) + 555))
    y = int(round(abs(y) + 505))
    
    return(x,y)

In [3]:
vrep_GUI(-4,-4)

(155, 905)

In [1]:
import cv2
import numpy 
import copy
import collections
import math
import time
import sys
import random


def Init_Map():

    img_size = [1011,1111]
    img = numpy.zeros([1011,1111,3])
    print(img.shape)

    img[:,:,0] = numpy.ones(img_size)*240.0
    img[:,:,1] = numpy.ones(img_size)*240.0
    img[:,:,2] = numpy.ones(img_size)*240.0

    img = cv2.circle(img,(150,180), 80, (0,0,0), -1)
    img = cv2.rectangle(img,(150,100),(310,260),(0,0,0),-1)
    img = cv2.circle(img,(310,180), 80, (0,0,0), -1)

    img = cv2.circle(img,(390,45), 40, (0,0,0), -1)
    img = cv2.circle(img,(438,274), 40, (0,0,0), -1)
    img = cv2.circle(img,(438,736), 40, (0,0,0), -1)
    img = cv2.circle(img,(390,965), 40, (0,0,0), -1)
    #img = cv2.circle(img,(555,505), 10, (0,0,0), -1)

    img = cv2.rectangle(img,(438,513),(529,696),(0,0,0),-1)
    img = cv2.rectangle(img,(529,660),(712,736),(0,0,0),-1)

    img = cv2.rectangle(img,(474,823),(748,975),(0,0,0),-1)
    img = cv2.rectangle(img,(685,975),(1110,1010),(0,0,0),-1)
    img = cv2.rectangle(img,(779,917),(896,975),(0,0,0),-1)
    img = cv2.rectangle(img,(927,899),(1110,975),(0,0,0),-1)

    img = cv2.rectangle(img,(832,0),(918,183),(0,0,0),-1)

    img = cv2.rectangle(img,(747,313),(1110,389),(0,0,0),-1)
    img = cv2.rectangle(img,(983,0),(1026,91),(0,0,0),-1)

    img = cv2.rectangle(img,(785,619),(937,736),(0,0,0),-1)
    img = cv2.rectangle(img,(1052,445),(1110,562),(0,0,0),-1)
    img = cv2.rectangle(img,(1019,562),(1110,648),(0,0,0),-1)
    img = cv2.rectangle(img,(1052,715),(1110,832),(0,0,0),-1)


    canny_img = cv2.Canny(numpy.uint8(img),100,200)
    indices = numpy.where(canny_img != [0])
    coordinates = zip(indices[0], indices[1])

    for i,j in coordinates:
        img = cv2.circle(img,(j,i), 20, (150,150,150), -1)

    cv2.namedWindow('RRL_Map.jpg', cv2.WINDOW_NORMAL)        # Create window with freedom of dimensions
    cv2.resizeWindow('RRL_Map.jpg', 700, 800)              

    #img = cv2.addWeighted(img_clone, 0.3, img, 0.7, 0)

    #img = cv2.flip( img, 0 )
    cv2.imwrite('RRL_Map.jpg', img)

Init_Map()

(1011, 1111, 3)


In [2]:
def Take_Starting_Node():
    
    img = cv2.imread('RRL_Map.jpg')
    gx = 150
    gy = 150
    while img[gy][gx][0] != 240:
        X = ((input("Give a Starting Node x in range(0,1110) and y in range(0,1010)   ")).split(','))
        #print(X)
        gx = int(X[0])
        gy = int(X[1])
    
        #gx,gy = Goal_Node
        if img[gy][gx][0] == 240:
            print('Given Starting Node is acceptable')
            img = cv2.circle(img,(gx,gy), 2, (0,0,255), -1)

            cv2.imwrite('RRL_Map.jpg', img)
            break
        else:
            print('Given Node is in Obstacle Space, please give a new node')
          
    return(int(X[0]),int(X[1]))

In [3]:
def Take_Goal_Node():
    img = cv2.imread('RRL_Map.jpg')
    
    gx = 150
    gy = 150
    while img[gy][gx][0] != 240:
        X = ((input("Give a Goal Node x in range(0,1110) and y in range(0,1010)   ")).split(','))
        #print(X)
        gx = int(X[0])
        gy = int(X[1])
    
        if img[gy][gx][0] in range(237,243):
            print('Given Goal Node is acceptable')
            img = cv2.circle(img,(gx,gy), 3, (0,255,0), -1)

            cv2.imwrite('RRL_Map.jpg', img)
            break
        else:
            print('Given Node is in Obstacle Space, please give a new node')
     
    return(int(X[0]),int(X[1]))

In [4]:
def Mark_Read(x,y):
    
    img = cv2.imread('RRL_Map.jpg')
    img = cv2.circle(img,(x,y), 3, (0,0,0), -1)
    cv2.imwrite('RRL_Map.jpg', img)
    cv2.imshow('RRL_Map.jpg', img)
    cv2.waitKey(1)

In [5]:
def Mark_Line(node1,node2):
    
    x1,y1 = node1
    x2,y2 = node2.child
    img = cv2.imread('RRL_Map.jpg')
    img = cv2.line(img, (x1,y1),(x2,y2), (0,0,0), 1)
    cv2.imwrite('RRL_Map.jpg', img)
    cv2.imshow('RRL_Map.jpg', img)
    cv2.waitKey(1)

In [6]:
class Node:

    def __init__(self, child, parent=None,cost = 0,costgo = 0, angle = 0, action = None):
        self.child = child
        self.parent = parent
        self.cost = cost
        self.costgo = costgo
        self.angle = angle
        self.action = action

In [7]:
def Node_Actions(node,v1,v2,th):
    
    img = cv2.imread('RRL_Map.jpg')
    
    dt = 1
    r = 3.8
    l = 25

    (x,y) = node
    directions = {'RR':((int(round(x + (r/2)*(v1)*math.cos(((r/l)*(v1)*dt)))),int(round(y - (r/2)*(v1)*math.sin(((r/l)*(v1)*dt))))),18.68, -(r/l)*(v1)*dt),
                  'lRR':((int(round(x + (r/2)*(v2)*math.cos(((r/l)*(v2)*dt)))),int(round(y - (r/2)*(v2)*math.sin(((r/l)*(v2)*dt))))),9.21, -(r/l)*(v2)*dt),
                  'RL':((int(round(x + (r/2)*(v1 + v2)*math.cos(((r/l)*(v1 - v2)*dt)))),int(round(y - (r/2)*(v1 + v2)*math.sin(((r/l)*(v1 - v2)*dt))))),28.42, -(r/l)*(v1 - v2)*dt),
                  'S' :((int(round(x + (r/2)*2*(v1)*dt)),y),38, 0),
                  'lS' :((int(round(x + (r/2)*2*(v2)*dt)),y),19, 0),
                  'LR':((int(round(x + (r/2)*(v1 + v2)*math.cos(((r/l)*(v2 - v1)*dt)))),int(round(y + (r/2)*(v1 + v2)*math.sin(((r/l)*(v1 - v2)*dt))))),28.42, (r/l)*(v1 - v2)*dt),
                  'lLL':((int(round(x + (r/2)*(v2)*math.cos(((r/l)*(v2)*dt)))),int(round(y + (r/2)*(v2)*math.sin(((r/l)*(v2)*dt))))),9.21, (r/l)*(v2)*dt),
                  'LL':((int(round(x + (r/2)*(v1)*math.cos(((r/l)*(v1)*dt)))),int(round(y + (r/2)*(v1)*math.sin(((r/l)*(v1)*dt))))),18.68, (r/l)*(v1)*dt)}
   
    valid_direc = {}
    for i in directions.items():
        d,((nx,ny),action_cost,a) = i
        nx, ny = int(round((nx-x)*math.cos((th)) - (ny-y)*math.sin((th)))) + x, int(round((ny-y)*math.cos((th)) + (nx-x)*math.sin((th)))) + y
        #print(nx,ny)
        if nx in range(0,1111) and ny in range(0,1011):
            #print(img[ny][nx])
            if img[ny][nx][0] in range(230,250):
#                 img = cv2.circle(img,(nx,ny), 3, (0,0,0), -1)
#                 cv2.imwrite('RRL_Map.jpg', img)
                valid_direc.update({(d,((nx,ny),action_cost,a))})
    
    return(valid_direc)
    

In [8]:
def closest_node(node, nodes):
    closest_index = distance.cdist([node], nodes).argmin()
    return nodes[closest_index]

In [9]:
def Steer(random_point, near_node):

        
        theta = math.atan2(random_point[1] - near_node[1], random_point[0] - near_node[0])
        newNode = Node((random_point[0], random_point[1]))
        #print('newNodes.child before steer',newNode.child)
        currentDistance = math.sqrt(
            (random_point[1] - near_node[1]) ** 2 + (random_point[0] - near_node[0]) ** 2)
        # Find a point within expandDis of nind, and closest to rnd
        if currentDistance <= 10:
            pass
        else:
            newNode.child = (int(near_node[0] + 10*math.cos(theta)), int(near_node[1] + 10*math.sin(theta)))
            
        newNode.cost = float("inf")
        #print('newNodes.child after steer',newNode.child)
        
        return newNode

In [10]:
def get_random_point():
    x = random.randint(0, 1110)
    y = random.randint(0, 1010)
    return (x,y)

In [11]:
def GetNearestNode(seen_nodes , randompoint):
    
    dmin = 9999999999
    for node in seen_nodes.values():
        if (node.child[0] - randompoint[0]) ** 2 + (node.child[1] - randompoint[1])** 2 < dmin:
            newnode = node.child
            dmin = (node.child[0] - randompoint[0]) ** 2 + (node.child[1] - randompoint[1])** 2

    return newnode

In [12]:
def choose_parent(newNode, near_nodes):
        if not near_nodes:
            return newNode
        #print(newNode)
        mincost = 999999999
        mincost_node = []
        for i in near_nodes:
            #print("i", i)
            dx = newNode.child[0] - i.child[0]
            dy = newNode.child[1] - i.child[1]
            d = math.sqrt(dx ** 2 + dy ** 2)
            #print('distance', d)
            theta = math.atan2(dy, dx)
            #if obstacle_line_check(i.child, theta, d):
                #print('First If')
            if i.cost + d < mincost:
                #print('Second If')
                mincost = i.cost + d
                mincost_node = i

        newNode.cost = mincost
        newNode.parent = mincost_node
        #print("parent1:", newNode.parent)

        return newNode

In [13]:
 def Rewire(newNode, nearinds):
        #print('rewire',nearinds)
    
        for i in nearinds:


            dx = newNode.child[0] - i.child[0]
            dy = newNode.child[1] - i.child[1]
            d = math.sqrt(dx ** 2 + dy ** 2)

            scost = newNode.cost + d

            if i.cost > scost:
                theta = math.atan2(dy, dx)
                #if obstacle_line_check(i.child, theta, d):
                i.parent = newNode
                i.cost = scost
        #print(nearinds)  

In [14]:
def find_near_nodes(newNode,seen_nodes):
    
        nnode = len(seen_nodes)
        r = 50.0 * math.sqrt((math.log(nnode+1) / nnode+1))
        #print('r',r)

        near_node_list = []
        for node in seen_nodes.values():
            #print(node.child)
            d = (node.child[0] - newNode.child[0]) ** 2 + (node.child[1] - newNode.child[1]) ** 2 
            if d < r**2:
                near_node_list.append(node)
                
        return near_node_list

In [15]:
def obstacle_check(new_node):
    #print('obs check',new_node)
    x,y = new_node
    img = cv2.imread('RRL_Map.jpg')
    
    #print('img',img[int(y)][int(x)])
    if img[int(y)][int(x)][0] in range(230,250):
        return False
    else:
        return True

In [16]:
def obstacle_line_check(node, theta, d):
    
    tmpNode = copy.deepcopy(node)
    #print(tmpNode)
    tmpNode = list(tmpNode)

    for i in range(int(round(d) / 0.5)):
        tmpNode[0] += 0.5 * math.cos(theta)
        tmpNode[1] += 0.5 * math.sin(theta)
        if not obstacle_check(tmpNode):
            return False

    return True

In [17]:
def costgo(Start,Goal):
    
    x,y = Start
    m,n = Goal
    return abs(m-x)+abs(n-y)

In [18]:
def Solve_RRTStar(Start,Goal,rePlan=False):
    
    Init_Map()
    xg,yg = Goal
    
    #start_cost_togo = costgo(Start,Goal)
    start_node = Node(Start,None,0,0)
    Mark_Read(start_node.child[0],start_node.child[1])
    
    flag = True
    queue = collections.deque([start_node])
    seen  = {}
    seen[str((queue[0].child))] = start_node
    
    
    # while queue and flag:
    for _ in range(3000):
        #print(seen)
        Prandom = get_random_point()
        near_node = GetNearestNode(seen, Prandom)
        #print('near node',near_node)
        new_node = Steer(Prandom, near_node)
        #print('new_node',new_node.child)
        
        if not obstacle_check(new_node.child):
            
            if (math.sqrt((new_node.child[0] - xg)**2 + (new_node.child[1] - yg)**2) <= 20):
                print('Goal Node Reched')
                Goal_Node = Node(new_node.child)
                near_nodes = find_near_nodes(new_node,seen)
                Goal_Node = choose_parent(new_node, near_nodes)
                Rewire(Goal_Node, near_nodes)
                Mark_Read(Goal_Node.child[0],Goal_Node.child[1])
                # flag = False
                if rePlan:
                    break
                
            near_nodes = find_near_nodes(new_node,seen)
#             for i in near_nodes:
#                 print('near_nodes',i.child)
            
            #print("new node:", new_node.child)
            new_node = choose_parent(new_node, near_nodes)
#             print('new node parent2',new_node.parent)
#             print('new node child2',new_node.child)
            Rewire(new_node, near_nodes)
            seen[(str(new_node.child))] = new_node
            Mark_Read(new_node.child[0],new_node.child[1])
#             for i in near_nodes:
#                 print('near_nodes2',i.child)
            
            #print("new node2:", new_node.child)
            Mark_Line(new_node.child, new_node.parent)
            
        
    P = []
    prev_node = Goal_Node.child
    P.append(Goal_Node.child)
    img = cv2.imread('RRL_Map.jpg')
    while Goal_Node.child != Start and Goal_Node.parent:
        #print('Goal_node parent',Goal_Node.parent)
        Goal_Node = Goal_Node.parent
        p,q = Goal_Node.child
        #print('Goal_node child',Goal_Node.child)
        P.append(Goal_Node.child)
        #print(P)
        
        img = cv2.circle(img,(p,q), 3, (0,255,0), -1)
        img = cv2.line(img,prev_node,(p,q),(0,255,0),1)
        prev_node = (p,q)
        cv2.imwrite('RRL_Map_path.jpg', img)
        cv2.imshow('RRL_Map.jpg', img)
        cv2.waitKey(1)
        img = cv2.imread('RRL_Map_path.jpg')

    cv2.destroyAllWindows

    P.reverse()

    return P

In [19]:
Start = Take_Starting_Node()
Goal = Take_Goal_Node()

Give a Starting Node x in range(0,1110) and y in range(0,1010)   155,905
Given Starting Node is acceptable
Give a Goal Node x in range(0,1110) and y in range(0,1010)   600,100
Given Goal Node is acceptable


In [22]:
def initObst(path,num=2):

    img = cv2.imread('RRL_Map.jpg')
    i=0
    ''' This stores current point of obstacle in first two positions and 
        the movement at each time step in other two '''
    obsts = numpy.empty((0,4))
    while i!=2:

        ind = random.randint(1,len(path)-1)
        obsX, obsY = path[ind]
        obsSX, obsSY = [int(obsX-(ind*0.5)),int(obsY-(ind*0.8))]
        if img[obsSY,obsSX,0] in range(230,255):

            obsts = numpy.append(obsts,[[obsSX,obsSY,0.5,0.8]],axis=0)
            # cv2.circle(img,(obsSX,obsSY),15,(0,255,255),-1)
            i+=1

    cv2.imwrite('RRL_Map_randObst.jpg',img)

    return obsts

In [20]:
path = Solve_RRTStar(Start,Goal)

(1011, 1111, 3)
Goal Node Reched
Goal Node Reched
Goal Node Reched
Goal Node Reched
Goal_node parent <__main__.Node object at 0x11fa42be0>
Goal_node child (592, 136)
Goal_node parent <__main__.Node object at 0x11fb624e0>
Goal_node child (573, 149)
Goal_node parent <__main__.Node object at 0x1169b6b00>
Goal_node child (543, 181)
Goal_node parent <__main__.Node object at 0x11fb5c7f0>
Goal_node child (537, 220)
Goal_node parent <__main__.Node object at 0x1169c6fd0>
Goal_node child (526, 245)
Goal_node parent <__main__.Node object at 0x11fb55c18>
Goal_node child (516, 275)
Goal_node parent <__main__.Node object at 0x11fa49128>
Goal_node child (485, 314)
Goal_node parent <__main__.Node object at 0x1169c6710>
Goal_node child (478, 321)
Goal_node parent <__main__.Node object at 0x1169c6048>
Goal_node child (448, 354)
Goal_node parent <__main__.Node object at 0x11fb3f978>
Goal_node child (437, 358)
Goal_node parent <__main__.Node object at 0x10cf144a8>
Goal_node child (414, 380)
Goal_node pare

In [ ]:
def moveObst(obsts, img):

    for obst in obsts:

        obst[0] = numpy.add(obst[0],obst[2])
        obst[1] = numpy.add(obst[1],obst[3])
        cv2.circle(img, (int(obst[0]),int(obst[1])),15,(0,255,255),-1)

    return obsts

In [24]:
def getGoal(path, ind, img):

    for i in range(ind,len(path)):

        if img[path[i][1],path[i][0],0] in range(230,250):
            return (path[i][0],path[i][1])

    return None

In [25]:
# def Init_Map2():

#     img_size = [1011,1111]
#     img = numpy.zeros([1011,1111,3])
#     print(img.shape)

#     img[:,:,0] = numpy.ones(img_size)*240.0
#     img[:,:,1] = numpy.ones(img_size)*240.0
#     img[:,:,2] = numpy.ones(img_size)*240.0

#     img = cv2.circle(img,(150,180), 80, (0,0,0), -1)
#     img = cv2.rectangle(img,(150,100),(310,260),(0,0,0),-1)
#     img = cv2.circle(img,(310,180), 80, (0,0,0), -1)

#     img = cv2.circle(img,(390,45), 40, (0,0,0), -1)
#     img = cv2.circle(img,(438,274), 40, (0,0,0), -1)
#     img = cv2.circle(img,(438,736), 40, (0,0,0), -1)
#     img = cv2.circle(img,(390,965), 40, (0,0,0), -1)
#     #img = cv2.circle(img,(555,505), 10, (0,0,0), -1)

#     img = cv2.rectangle(img,(438,513),(529,696),(0,0,0),-1)
#     img = cv2.rectangle(img,(529,660),(712,736),(0,0,0),-1)

#     img = cv2.rectangle(img,(474,823),(748,975),(0,0,0),-1)
#     img = cv2.rectangle(img,(685,975),(1110,1010),(0,0,0),-1)
#     img = cv2.rectangle(img,(779,917),(896,975),(0,0,0),-1)
#     img = cv2.rectangle(img,(927,899),(1110,975),(0,0,0),-1)

#     img = cv2.rectangle(img,(832,0),(918,183),(0,0,0),-1)

#     img = cv2.rectangle(img,(747,313),(1110,389),(0,0,0),-1)
#     img = cv2.rectangle(img,(983,0),(1026,91),(0,0,0),-1)

#     img = cv2.rectangle(img,(785,619),(937,736),(0,0,0),-1)
#     img = cv2.rectangle(img,(1052,445),(1110,562),(0,0,0),-1)
#     img = cv2.rectangle(img,(1019,562),(1110,648),(0,0,0),-1)
#     img = cv2.rectangle(img,(1052,715),(1110,832),(0,0,0),-1)
    
#     img = cv2.rectangle(img,(200,700),(300,800),(0,0,0),-1)


#     canny_img = cv2.Canny(numpy.uint8(img),100,200)
#     indices = numpy.where(canny_img != [0])
#     coordinates = zip(indices[0], indices[1])

#     for i,j in coordinates:
#         img = cv2.circle(img,(j,i), 20, (150,150,150), -1)

#     cv2.namedWindow('RRL_Map2.jpg', cv2.WINDOW_NORMAL)        # Create window with freedom of dimensions
#     cv2.resizeWindow('RRL_Map2.jpg', 700, 800)              

#     #img = cv2.addWeighted(img_clone, 0.3, img, 0.7, 0)

#     #img = cv2.flip( img, 0 )
#     cv2.imwrite('RRL_Map2.jpg', img)

# Init_Map2()


def execPath(path, obsts):

    img = cv2.imread('RRL_Map_randObst.jpg')
    sizy,sizx = img.shape[:2]
    op = cv2.VideoWriter('sim.avi',cv2.VideoWriter_fourcc(*'XVID'),1.0,(sizx,sizy))
    imgObst = img.copy()
    for i,point in enumerate(path):

        if i==0:
            cv2.circle(img,point, 3, (0,255,0), -1)
            prevPt = point
            continue

        if imgObst[point[1],point[0],0] in range(230,250):
            
            cv2.circle(img,point, 3, (0,255,0), -1)
            cv2.line(img,prevPt,point,(0,255,0),1)
            prevPt = point
            imgObst = img.copy()
            obsts = moveObst(obsts, imgObst)
            op.write(imgObst.copy())

        else:

            #Replan Triggered
            G = getGoal(path, i, imgObst)
            cv2.imwrite('RRL_Map.jpg',imgObst)
            rePath = Solve_RRTStar(path[i-1],G)
            for j, rePoint in enumerate(rePath):

                if j==0:
                    cv2.circle(img,rePoint, 3, (0,255,0), -1)
                    continue
                cv2.circle(img,rePoint, 3, (0,255,0), -1)
                cv2.line(img,rePath[j-1],rePoint,(0,255,0),1)

                imgObst = img.copy()
                obsts = moveObst(obsts, imgObst)
                op.write(imgObst.copy())
            prevPt = rePoint

    op.release()

    return


if __name__ == '__main__':

    Start = Take_Starting_Node()
    Goal = Take_Goal_Node()


    # In[23]:


    # path = Solve_RRTStar(Start,Goal)
    # with open('path.pkl','wb') as f:
    #     pickle.dump(path,f)

    with open('path.pkl','rb') as g:
        path = pickle.load(g)
    obsts = initObst(path)
    execPath(path, obsts)


Give a Starting Node x in range(0,1110) and y in range(0,1010)   155,905
Given Starting Node is acceptable
Give a Goal Node x in range(0,1110) and y in range(0,1010)   155,605
Given Goal Node is acceptable


FileNotFoundError: [Errno 2] No such file or directory: 'path.pkl'

In [ ]:
# def Solve_RRTStar2(Start,Goal):
    
#     Init_Map2()
#     xg,yg = Goal
    
#     #start_cost_togo = costgo(Start,Goal)
#     start_node = Node(Start,None,0,0)
#     Mark_Read(start_node.child[0],start_node.child[1])
    
#     flag = True
#     queue = collections.deque([start_node])
#     seen  = {}
#     seen[str((queue[0].child))] = start_node
    
# #     V = ((input("Give two wheel speeds   ")).split(','))
# #     v1 = int(V[0])
# #     v2 = int(V[1])
# #     angle = 0
    
#     while queue and flag:
#         #print(seen)
#         Prandom = get_random_point()
#         near_node = GetNearestNode(seen, Prandom)
#         #print('near node',near_node)
#         new_node = Steer(Prandom, near_node)
#         #print('new_node',new_node.child)
        
#         if not obstacle_check(new_node.child):
            
#             if (math.sqrt((new_node.child[0] - xg)**2 + (new_node.child[1] - yg)**2) <= 20):
#                 print('Goal Node Reached')
#                 Goal_Node = Node(new_node.child)
#                 near_nodes = find_near_nodes(new_node,seen)
#                 Goal_Node = choose_parent(new_node, near_nodes)
#                 Rewire(Goal_Node, near_nodes)
#                 Mark_Read(Goal_Node.child[0],Goal_Node.child[1])
#                 flag = False
#                 break
                
#             near_nodes = find_near_nodes(new_node,seen)
# #             for i in near_nodes:
# #                 print('near_nodes',i.child)
            
#             #print("new node:", new_node.child)
#             new_node = choose_parent(new_node, near_nodes)
# #             print('new node parent2',new_node.parent)
# #             print('new node child2',new_node.child)
#             Rewire(new_node, near_nodes)
#             seen[(str(new_node.child))] = new_node
#             Mark_Read(new_node.child[0],new_node.child[1])
# #             for i in near_nodes:
# #                 print('near_nodes2',i.child)
            
#             #print("new node2:", new_node.child)
#             Mark_Line(new_node.child, new_node.parent)
            
        
#     P = []
#     prev_node = Goal_Node.child
#     P.append(Goal_Node.child)
#     while Goal_Node.child != Start and Goal_Node.parent:
#         print('Goal_node parent',Goal_Node.parent)
#         Goal_Node = Goal_Node.parent
#         p,q = Goal_Node.child
#         print('Goal_node child',Goal_Node.child)
#         P.append(Goal_Node.child)
#         #print(P)
        
#         img = cv2.imread('RRL_Map.jpg')
#         img = cv2.circle(img,(p,q), 3, (0,255,0), -1)
#         img = cv2.line(img,prev_node,(p,q),(0,255,0),1)
#         prev_node = (p,q)
#         cv2.imwrite('RRL_Map.jpg', img)
#         cv2.imshow('RRL_Map.jpg', img)
#         cv2.waitKey(1)

#     cv2.destroyAllWindows

In [ ]:
# def Re_Plan_RRTstar(path):
#     img = cv2.imread('RRL_Map2.jpg')
#     prev_node = path.pop()
#     img = cv2.circle(img,(prev_node[0],prev_node[1]), 3, (0,255,0), -1)
#     path.reverse
#     for i in range(len(path)):
#         x,y = path[i]
#         if img[y][x][0] not in range(235,245):
#             x1,y1 = path[i]
#             while img[y1][x1][0] not in range(235,245):
#                 x1,y1 = path[i+1]
#                 i += 1
                
#             new_path = Solve_RRTStar2(prev_node,(x1,y1))
                
#         else:
#             img = cv2.circle(img,(x,y), 3, (0,0,0), -1)
#             img = cv2.line(img,prev_node,(x,y),(0,0,0),1)
#             cv2.imwrite('RRL_Map2.jpg', img)
#             cv2.imshow('RRL_Map2.jpg', img)
#             cv2.waitKey(1)
        
#         prev_node = x,y 
        
#     return path

In [ ]:
# path = Re_Plan_RRTstar(path)